# 사람처럼 보이기 

## 헤더 수정하기 
1. 헤더를 실제 브라우저처럼 만들자. requests 라이브러리로 가능하다. 
www.whatismybrowser.com 에서 브라우저가 보내는 헤더가 서버에서 어떻게 보이지는지 테스트할 수 있다.
2. 헤더 바꾸는걸로 크롤링하는 언어도 바꿀 수 있다. ( 대형 웹사이트는 ) 
3. 광고가 너무 많아서 짜증나면, 모바일버전으로 하면 줄어든다. (보통) , 다 없어지는 경우도 있고.


In [6]:

import requests
from bs4 import BeautifulSoup

session = requests.Session()
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
           'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'}
url = 'https://www.whatismybrowser.com/' \
      'developers/what-http-headers-is-my-browser-sending'
req = session.get(url, headers=headers)

bs = BeautifulSoup(req.text, 'html.parser')
print(bs.find('table', {'class':'table-striped'}).get_text)

<bound method PageElement.get_text of <table class="table table-striped table-data">
<tr>
<th>ACCEPT</th>
<td><span class="code detected_result">text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7</span></td>
</tr>
<tr>
<th>ACCEPT-ENCODING</th>
<td><span class="code detected_result">gzip, deflate</span></td>
</tr>
<tr>
<th>CONNECTION</th>
<td><span class="code detected_result">keep-alive</span></td>
</tr>
<tr>
<th>CONTENT-LENGTH</th>
<td><span class="code detected_result"></span></td>
</tr>
<tr>
<th>CONTENT-TYPE</th>
<td><span class="code detected_result"></span></td>
</tr>
<tr>
<th>HOST</th>
<td><span class="code detected_result">www.whatismybrowser.com</span></td>
</tr>
<tr>
<th>USER-AGENT</th>
<td><span class="code detected_result">Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36</span></td>
</tr>
</table>>


## 쿠키 조작 
1. 사이트가 생성하는 쿠키를 점검해봐야합니다.
쿠키로 인해 오히려 의심받을 수 있다.( 쿠키로 내 이동경로를 파악하고 있다면, 폼 완성 속도, 빠른 페이지 이동속도가 비정상적이라고 판단할 수 있음.)
2. EditThisCookie 플러그인으로 사이트에 방문함에 따라 쿠기가 어떻게 바뀌는지 보여주는 플러그인이 있다.

In [13]:
# requests 로는 자바스크립트 실행 못하니까 구글 애널리틱스 같은 추적 소프트웨어에서 만드는 쿠키는 처리하지못합니다.
# 보통 그런 쿠키는 클라이어언트의 실행을 마치거나, 버튼 클릭같은 이벤트로 생성됨.
# 따라서 셀레니움 필요함.

# get_cookies() : 쿠키 가져오기
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from chromedriver_autoinstaller import install
install()

chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

driver.get('http://pythonscraping.com')
driver.implicitly_wait(1)
print(driver.get_cookies())

[{'domain': '.pythonscraping.com', 'expiry': 1756803633, 'httpOnly': False, 'name': '_ga', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GA1.1.1341282419.1722243634'}, {'domain': '.pythonscraping.com', 'expiry': 1756803633, 'httpOnly': False, 'name': '_ga_G60J5CGY1N', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GS1.1.1722243633.1.0.1722243633.0.0.0'}]


In [ ]:
# 쿠키는 복사해서 사용하자.
# 이러면  구글 애널리스틱은 첫번째 드라이버와 두 번째 드라이버가 같은거라고 판단할꺼다. 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=chrome_options)

driver.get('http://pythonscraping.com')
driver.implicitly_wait(1)

savedCookies = driver.get_cookies()
print(savedCookies)

driver2 = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=chrome_options)

driver2.get('http://pythonscraping.com')
driver2.delete_all_cookies()
for cookie in savedCookies:
    driver2.add_cookie(cookie)

driver2.get('http://pythonscraping.com')
driver.implicitly_wait(1)
print(driver2.get_cookies())



# 타이밍이 가장 중요하다.
time.sleep(3) 이거 하나 넣으면 "나는 로봇이 아닙니다." 이거 안 해도됨 ㅋㅋ-ㅋ

# 폼 보안 기능 우회하기
- 잘 보호된거는 CAPTCHA 사용
- 헤더와 IP주소는 나중에.. 

# 숨긴 필드 값 
## type="hidden" 로 막는다.

1. 방법1: 폼을 생성할 때 무작위로 변수를 만들어 폼에 넣고 그 값을 폼을 처리하는 페이지에서 받는 방법.(폼을 만들 때 무작위로 생성된 변수를 넣어놓고, 폼을 처리하는 페이지에서 그 변수를 받아서 비교하는 방법)
=> 먼저 폼 페이지 스크랩해서 무작위로 생성된 변수 가져온 후 처리페이지로 보내면됨
2. 허니팟 honey pot : 폼을 보낼 때, 숨겨진 필드에 값이 들어가있으면, 로봇으로 판단한다. (그러니까 함정을 파는거네요. 실제 서비스는 email필요없는데 email address 란에 이메일을 넣어서 보내면, 로봇으로 판단하는거죠.)


In [ ]:
# 2, 허니팟 피하기 
# is_displayed() 매서드로 확인하자! 그래야 내가 봇인지 안 들킨다구
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH))

driver.get('http://pythonscraping.com/pages/itsatrap.html')
links = driver.find_elements(By.TAG_NAME, 'a')
for link in links:
    if not link.is_displayed():
        print(f'The link {link.get_attribute("href")} is a trap')

fields = driver.find_elements(By.TAG_NAME, 'input')
for field in fields:
    if not field.is_displayed():
        print(f'Do not change value of {field.get_attribute("name")}')